In [1]:

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
import sqlite3
from sklearn.metrics import mean_squared_error, r2_score


In [2]:

# Path to SQLite database
db_path = 'database/stocks_data.db'

# Load data from SQLite
with sqlite3.connect(db_path) as conn:
    query = "SELECT * FROM processed_stocks"
    data = pd.read_sql(query, conn)
print(f"Loaded processed data: {data.shape[0]} rows")


Loaded processed data: 107322 rows


In [3]:
# Step 1: Set default ticker
default_ticker = 'XOM'

# Step 2: Filter data for the default ticker
ticker_data = data[data['Ticker'] == default_ticker]
print(f"Loaded data for {default_ticker}: {ticker_data.shape[0]} rows")

Loaded data for XOM: 18803 rows


In [4]:

# Define features and target
features = ['7-day MA', '14-day MA', 'Volatility', 'Lag_1', 'Lag_2']
target = 'Adj Close'

X = ticker_data[features]
y = ticker_data[target]


In [5]:

# # Drop missing values
# data = data.dropna(subset=features + [target])
# X = data[features].values
# y = data[target].values

# # Normalize the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# # Reshape target to 2D for compatibility
# y = y.reshape(-1, 1)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [6]:

# Reshape input for LSTM (samples, timesteps, features)
# scaler.MinMaxScaler()
X_train_scaled = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_scaled = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])


In [7]:

# Build and train the LSTM model
model = Sequential()
model.add(LSTM(32, input_shape=(1, X_train_scaled.shape[2]), activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Early stopping for training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train_scaled, y_train, epochs=100, batch_size=42, validation_data=(X_test_scaled, y_test), callbacks=[early_stopping])

Epoch 1/100


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


359/359 ━━━━━━━━━━━━━━━━━━━━ 1s 773us/step - loss: 2780.5342 - val_loss: 1750.7427
Epoch 2/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 506us/step - loss: 1170.7338 - val_loss: 114.7116
Epoch 3/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 567us/step - loss: 54.0509 - val_loss: 6.3664
Epoch 4/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - loss: 5.9899 - val_loss: 5.7398
Epoch 5/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 511us/step - loss: 4.7897 - val_loss: 5.3762
Epoch 6/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - loss: 3.7579 - val_loss: 5.0543
Epoch 7/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step - loss: 3.3552 - val_loss: 4.8064
Epoch 8/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step - loss: 3.9971 - val_loss: 4.5985
Epoch 9/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step - loss: 5.8093 - val_loss: 4.3967
Epoch 10/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step - loss: 3.4389 - val_loss: 4.2080
Epoch 11/100
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - loss: 4.5840 - val_loss: 4.0392
Epoch 12/100
359/35

In [8]:

# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R-squared (R²): {r2:.2f}")

118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 662us/step
Model Evaluation:
Mean Squared Error (MSE): 1.63
R-squared (R²): 1.00


In [9]:
# Step 8: Save the trained model and scaler
model_filename = f'models/model_{default_ticker}.pkl'
scaler_filename = f'models/scaler_{default_ticker}.pkl'

joblib.dump(model, model_filename)
joblib.dump(scaler, scaler_filename)

print(f"{default_ticker} model saved as '{model_filename}'")
print(f"{default_ticker} scaler saved as '{scaler_filename}'")

XOM model saved as 'models/model_XOM.pkl'
XOM scaler saved as 'models/scaler_XOM.pkl'
